In [1]:
import json
from datasets import load_dataset, Dataset
# Specify the path to the JSON file
file_path = 'datasets/squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


In [2]:
sentence1 = []
sentence2 = []
data_for_eval = {
    'id': [],
    'title': [],
    'context': [],
    'question': [],
    'answers': []
}
i = 1
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            data_for_answer = {
                "text": [],
                'answer_start': []
            }
            for answer in qa_pair['answers']:
                short_answer = answer['text']
                answer_start = answer['answer_start']
                sentence1.append(question)
                sentence2.append(short_answer)
                data_for_answer['text'].append(short_answer)
                data_for_answer['answer_start'].append(answer_start)
            data_for_eval['id'].append(str(i))
            data_for_eval['title'].append(title)
            data_for_eval['context'].append(context)
            data_for_eval['question'].append(question)
            data_for_eval['answers'].append(data_for_answer)
            i += 1
                # print(answer)
dataset_for_eval = Dataset.from_dict(data_for_eval)

In [3]:
from evaluate import evaluator
task_evaluator = evaluator("question-answering")
# data = load_dataset("squad_v2", split="validation[:2]")
results = task_evaluator.compute(
    model_or_pipeline="classla/bcms-bertic",
    data=dataset_for_eval,
    metric="squad_v2",
    squad_v2_format=True,
)

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/selena/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
results

{'exact': 0.0,
 'f1': 4.2324445484296795,
 'total': 269,
 'HasAns_exact': 0.0,
 'HasAns_f1': 4.2324445484296795,
 'HasAns_total': 269,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 4.232444548429679,
 'best_f1_thresh': 0.00017354990995954722,
 'total_time_in_seconds': 56.48018586899707,
 'samples_per_second': 4.762732201765976,
 'latency_in_seconds': 0.20996351624162482}

loading the data to wanted format

In [5]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)
dataset_for_eval = Dataset.from_dict(data_for_eval)

In [10]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'cotext' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'query' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['query'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [11]:
dataset_for_eval_df.to_csv('datasets/dataset_procesed.csv')


In [15]:
dataset_for_eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18896 entries, 0 to 18895
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   18896 non-null  object
 1   cotext  18896 non-null  object
 2   query   18896 non-null  object
dtypes: object(3)
memory usage: 443.0+ KB


making the pipieline for all models

In [14]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import pandas as pd
import numpy as np

# Example data (replace with your actual data)
titles = ["University_of_Notre_Dame", "Normans"]
contexts = [
    "Arhitektonski, škola ima katolički karakter...",
    "The Normans (Norman: Nourmands; French: Normands...)"
]
questions = [
    "Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?",
    "In what country is Normandy located?"
]
answers = [
    {"text": ["Svetoj Bernadet Subiru"], "answer_start": [464]},
    {"text": ["France"], "answer_start": [159]}
]

# Prepare the dataset
data_for_eval = {
    'id': [str(i) for i in range(len(titles))],
    'title': titles,
    'context': contexts,
    'question': questions,
    'answers': answers
}

dataset_for_eval = Dataset.from_dict(data_for_eval)

# Load model and tokenizer
model_name = "classla/bcms-bertic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Generate predictions
def generate_predictions(dataset):
    predictions = []
    for example in dataset:
        prediction = qa_pipeline(question=example['question'], context=example['context'])
        predictions.append({
            'id': example['id'],
            'prediction_text': prediction['answer'],
            'no_answer_probability': prediction.get('score', 0.0)
        })
    return predictions

predictions = generate_predictions(dataset_for_eval)

# Custom evaluation metrics
def compute_custom_metrics(predictions, references):
    # Example metric: Exact Match (EM) and F1 Score
    # def exact_match_score(prediction, ground_truth):
    #     return int(prediction == ground_truth)

    def f1_score(prediction, ground_truth):
        prediction_tokens = prediction.split()
        ground_truth_tokens = ground_truth.split()
        common = set(prediction_tokens) & set(ground_truth_tokens)
        if len(common) == 0:
            return 0
        precision = len(common) / len(prediction_tokens)
        recall = len(common) / len(ground_truth_tokens)
        f1 = 2 * (precision * recall) / (precision + recall)
        return f1

    em = []
    f1 = []
    for pred, ref in zip(predictions, references):
        ground_truth_text = ref['text'][0]  # assuming there's only one ground truth answer
        em.append(exact_match_score(pred['prediction_text'], ground_truth_text))
        f1.append(f1_score(pred['prediction_text'], ground_truth_text))
    
    return {
        'exact_match': np.mean(em),
        'f1_score': np.mean(f1)
    }

# References from the dataset
references = [{'id': example['id'], 'text': example['answers']['text']} for example in dataset_for_eval]

# Compute custom metrics
results = compute_custom_metrics(predictions, references)

print(results)


/home/selena/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'exact_match': np.float64(0.0), 'f1_score': np.float64(0.0)}


In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import numpy as np
from datasets import Dataset
import pytrec_eval


# Example data (replace with your actual data)
titles = ["University_of_Notre_Dame", "Normans"]
contexts = [
    "Arhitektonski, škola ima katolički karakter...",
    "The Normans (Norman: Nourmands; French: Normands...)"
]
questions = [
    "Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?",
    "In what country is Normandy located?"
]

# Assume these are the ground truth answers for evaluation purposes
reference_answers = [
    "Svetoj Bernadet Subiru",
    "France"
]

# Prepare the dataset
data_for_eval = {
    'id': [str(i) for i in range(len(titles))],
    'title': titles,
    'context': contexts,
    'question': questions
}

dataset_for_eval = Dataset.from_dict(data_for_eval)

# Load model and tokenizer
model_name = "classla/bcms-bertic"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# Create model embeddings
# def generate_embeddings(text):
# Initialize the question-answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)
# Measuring query-context similarity
# def compute_similarity(query, context, top_k):
# Generate predictions
def generate_predictions(dataset):
    predictions = []
    for example in dataset:
        prediction = qa_pipeline(question=example['question'], context=example['context'])
        predictions.append({
            'id': example['id'],
            'prediction_text': prediction['answer']
        })
    return predictions

predictions = generate_predictions(dataset_for_eval)

# Custom evaluation metrics
def compute_custom_metrics(predictions, references):
    # Example metric: Exact Match (EM) and F1 Score
    def precision_k(prediction, ground_truth, k):
        evaluator = pytrec_eval.RelevanceEvaluator(ground_truth, {'P.4'})
        results = evaluator.evaluate(prediction)
        return results

    def f1_score(prediction, ground_truth):
        prediction_tokens = prediction.split()
        ground_truth_tokens = ground_truth.split()
        common = set(prediction_tokens) & set(ground_truth_tokens)
        if len(common) == 0:
            return 0
        precision = len(common) / len(prediction_tokens)
        recall = len(common) / len(ground_truth_tokens)
        f1 = 2 * (precision * recall) / (precision + recall)
        return f1

    em = []
    f1 = []
    for pred, ref in zip(predictions, references):
        em.append(exact_match_score(pred['prediction_text'], ref))
        f1.append(f1_score(pred['prediction_text'], ref))
    
    return {
        'exact_match': np.mean(em),
        'f1_score': np.mean(f1)
    }

# Compute custom metrics
results = compute_custom_metrics(predictions, reference_answers)

print(results)


/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'exact_match': np.float64(0.0), 'f1_score': np.float64(0.0)}
